## Crawling all NTU Researchers
- Just in case information from other faculties is required in later stages, all 1441 researchers from ntu were crawled with scrapy spider
- Using a crawler is also much faster
- output saved to researchers.json


In [1]:
import scrapy
import re
class CrawlResearchersSpider(scrapy.Spider):
    name = 'crawl_researchers'
    allowed_domains = ['dr.ntu.edu.sg']
    
    # start_urls = ["https://dr.ntu.edu.sg/simple-search?query=&location=researcherprofiles&crisID=&relationName=&sort_by=score&order=desc&rpp=50&etal=0&start=0"]
    start_urls = ["https://dr.ntu.edu.sg/simple-search?location=researcherprofiles"]
    
    def parse(self, response):
    
        # parse the all names and emails, then go to each profile
        domain = "https://dr.ntu.edu.sg"
        researcher_rows = response.css("table.table.table-hover td")
        names = researcher_rows.css("td[headers=t1] a::text").getall()
        emails = researcher_rows.css("td[headers=t3]::text").getall()
        endpoints = researcher_rows.css("td a::attr(href)").getall()
        for i, endpoint in enumerate(endpoints):
            dr_ntu_profile_url = domain+endpoint
            meta = {
                "full_name":names[i],
                "email":emails[i],
                "dr_ntu_url": dr_ntu_profile_url
            }
            yield response.follow(dr_ntu_profile_url, callback = self.parse_profile, meta = meta)
            
        # check if there's next page then go to next page
        pagination = response.css("ul.pagination.pull-right")
        has_next = "next" in pagination.css("li a[href] ::text").getall()
        if has_next:
            next_endpoint = pagination.css("li a::attr(href)").getall()[-1]
            next_link = domain+next_endpoint
            yield response.follow(next_link, callback=self.parse)
        
    # parse the schools and personal sites, then save following data: full_name, email, dr_ntu_url, schools, personal_site
    def parse_profile(self, response):
        output = {
            "full_name": response.meta["full_name"],
            "email":response.meta["email"],
            "dr_ntu_url": response.meta["dr_ntu_url"]
        }
        info = response.css("div[id=researcherInfo]")
        schools = [self.clean_school(string) for string in info.css("div::text").getall() if re.search('[a-zA-Z]', string)]
        output["schools"] = schools
        output["personal_site"] = info.css("div[id=personalsiteDiv] a::attr(href)").get()
        yield output
        
        
    # remove extra spaces, \n and \t
    def clean_school(self, string):
        string = re.sub(r"[\n\t]*", "", string)
        string = re.sub(' +', ' ', string)
        return string.strip()

## Extracting SCSE Researchers
- Used fuzzy string matching to acquire 82 scse researchers from 1441 ntu researchers
- although a simple solution could already suffice here, this proposed solution will be much more robust when the data becomes very dirty

In [2]:
import json
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 1\researchers\researchers.json"
with open(path) as f:
    data = json.load(f)
    data = pd.DataFrame(data)

In [5]:
# this scorer combines multiple scorers from fuzzwuzzy st "School of Computer Science and Engineering" can be matched with
# 1. "John, School  of Computer    Science   and  Engineering"
# 2. "John, School  of of Computer    Science   and  Engineering (SCSE)"
# 3. "John, School  of Computer    Engineering and Computer Science"
# Benefits: 
# minimal processing is required while still maintaining performance. 
# The problem with applying to much processing is that there are a lot of edge cases to account for, which is not feasible when the data size is very large
# this method is generalizable to other schools like "School of Physical & Mathematical Sciences" or even other strings
# will be advantageous when the data starts to become very dirty 
# Note: Actually a simple "Computer Science" in s2 will suffice in this problem because the data is still quite clean
def score(s1, s2):
    scores = [
        fuzz.token_sort_ratio(s1, s2), 
        fuzz.token_set_ratio(s1, s2), 
        fuzz.partial_ratio(s1, s2)
    ]
    return max(scores)

# return True if target_string is found in choices using the scorer
# Given choices ["Associate Chair (Faculty), School of Computer Science and Engineering (SCSE)", 'Associate Chair (Faculty), CCEB', 'Professor, School of Chemistry, Chemical Engineering and Biotechnology'
# "School of Computer Science and Engineering" will output True 
def fuzzy_in_list(target_string, choices, scorer = score, score_cutoff = 100):
    # this function by default, does the essential processing e.g leading, trailing spaces, non-letters, cases
    result = process.extractOne(target_string, choices, scorer = scorer, score_cutoff = score_cutoff)
    return result is not None

In [6]:
# indeed there are 82 researchers in SCSE (shown in website)
target_school = "School of Computer Science and Engineering"
in_scse = data.schools.apply(lambda choices: fuzzy_in_list(target_school, choices))
scse = data[in_scse]
scse

,full_name,email,dr_ntu_url,schools,personal_site
19,Lin Guosheng,gslin@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00483,"[Assistant Professor, School of Computer Scien...",https://guosheng.github.io/
22,Yu Han,han.yu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00334,"[Nanyang Assistant Professor, School of Comput...",http://hanyu.sg
26,Liu Weichen,liu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00280,"[Nanyang Assistant Professor, School of Comput...",https://personal.ntu.edu.sg/liu/
41,Lam Siew Kei,assklam@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00901,"[Associate Professor, School of Computer Scien...",https://personal.ntu.edu.sg/assklam/
46,Loke Yuan Ren,yrloke@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00585,"[Lecturer, School of Computer Science and Engi...",https://personal.ntu.edu.sg/yrloke/
...,...,...,...,...,...
1346,Long Cheng,c.long@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00772,"[Assistant Professor, School of Computer Scien...",https://personal.ntu.edu.sg/c.long
1368,Luo Siqiang （骆思强）,siqiang.luo@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01311,"[Assistant Professor, School of Computer Scien...",https://siqiangluo.wixsite.com/homepage
1391,Tang Xueyan,asxytang@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00075,"[Associate Professor, School of Computer Scien...",https://personal.ntu.edu.sg/asxytang/
1407,Erik Cambria,cambria@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00927,"[Associate Professor, School of Computer Scien...",https://sentic.net


In [7]:
# import pickle
# path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 1\researchers_names.pkl"
# with open(path, "wb") as f:
#     pickle.dump(data[in_scse].full_name.tolist(), f)

In [8]:
scse.personal_site[scse.personal_site.apply(lambda site: site is None)] = pd.NA
names = scse.full_name.tolist()

## dblp links
- subsequent sections no longer used crawler because it seems like the sites restricted too much crawling, thus i had to settle with beautifulsoup which is much slower
- steps:
    - perform search with the name
        - if there are no results try another variation of the name
    - if there is one result only, i assume its correct and return
    - if there are multiple
        - return the first one that matches "Nanyang Technological University"
        - if no match return the first result

In [9]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm

In [10]:
def get_dblp_link(name):
    soup  = get_soup(name)
    authors = soup.find("div", {"id":"completesearch-authors"}).find_all("li", {"itemtype":"http://schema.org/Person"})
    if len(authors) == 0:
        # if no results clean the name, try again
        soup  = get_soup(ugly_clean(name))
        authors = soup.find("div", {"id":"completesearch-authors"}).find_all("li", {"itemtype":"http://schema.org/Person"})
    # save the first link as the most likely solution
    if len(authors) >= 1:
        first_link = authors[0].find("a")["href"]
    # if there's only one option, return link
    if len(authors) == 1:
        return first_link
    # if there are multiple, check if its ntu, then return link 
    for a in authors:
        description = a.find("small")
        if description is not None and "Nanyang Technological University" in description.text:
            link = a.find("a")["href"]
            return link
    # if nothing fits, return first link
    return first_link

def get_soup(name):
    url = get_dblp_query_url(name)
    r = requests.get(url)
    soup  = BeautifulSoup(r.text, "html.parser")
    return soup
def get_dblp_query_url(name):
    return "https://dblp.org/search?q="+"%20".join(name.strip().split())
def ugly_clean(name):
    if "," in name:
        return name.split(",")[0]
    words = name.split()
    words[1] = words[1][0]
    return " ".join(words)

In [11]:
dblp_links = []
for name in tqdm(names):
    print(name)
    link = get_dblp_link(name)
    dblp_links.append(link)

  0%|                                                                                           | 0/82 [00:00<?, ?it/s]

Lin Guosheng


  1%|█                                                                                  | 1/82 [00:00<01:08,  1.18it/s]

Yu Han


  2%|██                                                                                 | 2/82 [00:02<01:24,  1.06s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████  | 80/82 [01:33<00:02,  1.07s/it]

Erik Cambria


 99%|█████████████████████████████████████████████████████████████████████████████████ | 81/82 [01:34<00:01,  1.25s/it]

Liu Yang


100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [01:35<00:00,  1.17s/it]


In [12]:
scse["dblp_links"] = dblp_links

C:\Users\tanch\AppData\Local\Temp\ipykernel_10728\4089554922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scse["dblp_links"] = dblp_links


In [14]:
scse

,full_name,email,dr_ntu_url,schools,personal_site,dblp_links
19,Lin Guosheng,gslin@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00483,"[Assistant Professor, School of Computer Scien...",https://guosheng.github.io/,https://dblp.org/pid/126/4778
22,Yu Han,han.yu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00334,"[Nanyang Assistant Professor, School of Comput...",http://hanyu.sg,https://dblp.org/pid/35/1096-1
26,Liu Weichen,liu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00280,"[Nanyang Assistant Professor, School of Comput...",https://personal.ntu.edu.sg/liu/,https://dblp.org/pid/24/914
41,Lam Siew Kei,assklam@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00901,"[Associate Professor, School of Computer Scien...",https://personal.ntu.edu.sg/assklam/,https://dblp.org/pid/74/1907
46,Loke Yuan Ren,yrloke@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00585,"[Lecturer, School of Computer Science and Engi...",https://personal.ntu.edu.sg/yrloke/,https://dblp.org/pid/11/9550
...,...,...,...,...,...,...
1346,Long Cheng,c.long@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00772,"[Assistant Professor, School of Computer Scien...",https://personal.ntu.edu.sg/c.long,https://dblp.org/pid/49/225
1368,Luo Siqiang （骆思强）,siqiang.luo@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01311,"[Assistant Professor, School of Computer Scien...",https://siqiangluo.wixsite.com/homepage,https://dblp.org/pid/117/5965
1391,Tang Xueyan,asxytang@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00075,"[Associate Professor, School of Computer Scien...",https://personal.ntu.edu.sg/asxytang/,https://dblp.org/pid/23/2460
1407,Erik Cambria,cambria@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00927,"[Associate Professor, School of Computer Scien...",https://sentic.net,https://dblp.org/pid/80/7421


## Extract number of citations by each scse researcher
- this section also doesn't use crawler because of crawling restriction
- Num citations are extracted from google scholar e.g https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=Erik+Cambria+ntu&btnG=
- steps are similar to the previous section

In [16]:
def get_citations(name):
    soup  = get_soup(name)
    authors = soup.find("div",{"id":"gsc_sa_ccl"}).find_all("div",{"class":"gsc_1usr"})
    if len(authors) == 0:
        # if no results clean the name, try again
        soup  = get_soup(ugly_clean(name))
        authors = soup.find("div",{"id":"gsc_sa_ccl"}).find_all("div",{"class":"gsc_1usr"})
        
    # save the first as the most likely solution
    first_cited_by = None
    if len(authors) >= 1:
        first_cited_by = authors[0].find("div",{"class":"gs_ai_cby"})
        first_cited_by = first_cited_by.text if first_cited_by is not None else ""
    # if there's only one option, return 
    if len(authors) == 1:
        return first_cited_by
    # if there are multiple, check if its ntu, then return  
    for a in authors:
        school = a.find("div",{"class":"gs_ai_aff"})
        school = school.text if school is not None else ""
        verified_at = a.find("div",{"class":"gs_ai_eml"})
        verified_at = verified_at.text if verified_at is not None else ""
        if "Nanyang Technological University" in school or "ntu.edu.sg" in verified_at:
            cited_by = a.find("div",{"class":"gs_ai_cby"})
            cited_by = cited_by.text if cited_by is not None else None
            return cited_by
    # if nothing fits, return first 
    return first_cited_by if first_cited_by is not None else None

def get_soup(name):
    url = get_google_scholar_url(name)
    r = requests.get(url)
    soup  = BeautifulSoup(r.text, "html.parser")
    return soup
def get_google_scholar_url(name, additional_keywords = ["ntu"]):
    keywords = name.strip().split()+additional_keywords
    return "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors="+"+".join(keywords)

In [17]:
import re
citations = []
for name in tqdm(names):
    name = re.sub(r'[^a-zA-Z ]', '', name)
    print(name)
    c = get_citations(name)
    citations.append(c)

  0%|                                                                                           | 0/82 [00:00<?, ?it/s]

Lin Guosheng


  1%|█                                                                                  | 1/82 [00:00<00:53,  1.52it/s]

Yu Han


  2%|██                                                                                 | 2/82 [00:03<02:29,  1.87s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████  | 80/82 [03:03<00:07,  3.73s/it]

Erik Cambria


 99%|█████████████████████████████████████████████████████████████████████████████████ | 81/82 [03:05<00:03,  3.10s/it]

Liu Yang


100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [03:08<00:00,  2.30s/it]


In [43]:
def clean_citations(string):
    if string is None or string == "":
        return pd.NA
    return int(string.split()[-1])

In [44]:
scse['num_citations'] = list(map(clean_citations, citations))

C:\Users\tanch\AppData\Local\Temp\ipykernel_10728\2880711355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scse['num_citations'] = list(map(clean_citations, citations))


## output
- sort by emails

In [51]:
cols = ["full_name","email","dr_ntu_url","personal_site", "dblp_links", "num_citations"]
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\solution.csv"
scse = scse.sort_values("email")
scse[cols]

,full_name,email,dr_ntu_url,personal_site,dblp_links,num_citations
1342,Wai Kin Adams Kong,adamskong@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00834,<NA>,https://dblp.org/pid/16/3792,<NA>
1240,Luu Anh Tuan,anhtuan.luu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01296,https://tuanluu.github.io/,https://dblp.org/pid/81/8329,2936
217,Anupam Chattopadhyay,anupam@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01076,https://scholar.google.co.in/citations?user=TI...,https://dblp.org/pid/99/4535,4875
76,Anwitaman Datta,anwitaman@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00706,https://personal.ntu.edu.sg/anwitaman/,https://dblp.org/pid/d/AnwitamanDatta,7669
754,Arvind Easwaran,arvinde@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00687,https://cps-research-group.github.io/,https://dblp.org/pid/73/1708,2506
...,...,...,...,...,...,...
46,Loke Yuan Ren,yrloke@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00585,https://personal.ntu.edu.sg/yrloke/,https://dblp.org/pid/11/9550,13
857,Zhang Jie,zhangj@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00759,https://personal.ntu.edu.sg/zhangj/,https://dblp.org/pid/84/6889-2,10287
1225,Zinovi Rabinovich,zinovi@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00214,zinovi.net,https://dblp.org/pid/93/4009,941
119,Liu Ziwei,ziwei.liu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00203,https://liuziwei7.github.io/,https://dblp.org/pid/05/6300-2,18681


In [53]:
scse[cols].to_csv(path, index = False)

## Other attempts:
- attempts at crawling dblp url and citations had very fast executions but debugging is slower and eventually there were site restrictions so BeautfulSoup was used instead

In [ ]:
import scrapy
import pickle
import time

    

class CrawlDblpResearchersSpider(scrapy.Spider):
    name = 'crawl_dblp_researchers'
    allowed_domains = ['dblp.org']
    start_urls = ["https://scholar.google.com/schhp?hl=en"]
    
    # path to raw researcher names 
    path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 1\researchers_names.pkl"
    
    
    def parse(self, response):
        # for each researcher name make a request
        names = self.get_researcher_names(CrawlDblpResearchersSpider.path)
        for name in names:
            link = self.get_dblp_link(name)
            meta = {
                "full_name":name,
                "link":link
            }
            print(meta["link"])
            yield response.follow(link, callback = self.parse_dblp, meta = meta)
            time.sleep(0.5)
            
    def parse_dblp(self, response):
        output = {
            "full_name":response.meta["full_name"],
            "link":response.meta["link"]
        }
        authors = response.css("div[id=completesearch-authors] ul.result-list")
        if len(authors)==1:
            dblp_link = authors.css("a::attr(href)").get()
            output["dblp_link"] = dblp_link
            output["multiple_results"] = False
            yield output
        else:
            for author in authors:
                description = " ".join(author.css("small::text").getall())
                if "Nanyang Technological University" in description:
                    dblp_link = author.css("a::attr(href)").get()
                    output["dblp_link"] = dblp_link
                    output["multiple_results"] = True
                    yield output
       
    def get_researcher_names(self, path):
        with open(path, "rb") as f:
            names = pickle.load(f)   
        return names
    def get_dblp_link(self, name):
        return "https://dblp.org/search?q="+"%20".join(name.strip().split())



In [ ]:
import scrapy
import pickle


class CrawlResearcherCitationsSpider(scrapy.Spider):
    name = 'crawl_researcher_citations'
    allowed_domains = ['scholar.google.com']
    start_urls = ['http://scholar.google.com/']

    # path to raw researcher names 
    path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 1\researchers_names.pkl"
    
    def parse(self, response):
        # for each researcher name make a request
        names = self.get_researcher_names(CrawlResearcherCitationsSpider.path)
        for name in names:
            link = self.get_google_scholar_link(name, additional_keywords = ["ntu"])
            meta = {
                "full_name":name,
                "link":link
            }
            
            yield response.follow(link, callback = self.parse_google_scholar, meta = meta)
    
    # 
    def parse_google_scholar(self, response):
        output = {
            "full_name":response.meta["full_name"],
            "link":response.meta["link"]
        }
        users = response.css("div[class=gsc_1usr]")
       
        for user in users:
            verified_at = user.css("div.gs_ai_eml span::text").get()
            organization = user.css("div.gs_ai_aff *::text").get()
            if verified_at=="ntu" or "Nanyang Technological University" in organization or "Singapore" in organization or "NTU" in organization:
                cited_by = user.css("div.gs_ai_cby::text").get()
                output["organization"] = organization
                output["verified_at"] = verified_at
                output["cited_by"] = cited_by
                yield output
                
                
                
    def get_researcher_names(self, path):
        with open(path, "rb") as f:
            names = pickle.load(f)   
        return names
    # add an addition "ntu" for greater accuracy
    def get_google_scholar_link(self, name, additional_keywords = []):
        keywords = name.strip().split()+additional_keywords 
        return "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors="+"+".join(keywords)

